# Imports

In [23]:
from maze_dataset.tokenization import (
    ALL_TOKENIZERS,
    MazeTokenizer,
    MazeTokenizer2,
    TokenizerElement,
    AdjListTokenizers,
    CoordTokenizers,
    PathTokenizers,
    PromptSequencers,
    TargetTokenizers,
)

from maze_dataset import (
    SPECIAL_TOKENS,
    VOCAB,
    VOCAB_LIST,
    VOCAB_TOKEN_TO_INDEX,
    LatticeMaze,
    LatticeMazeGenerators,
    MazeDataset,
    MazeDatasetConfig,
)


# `MazeTokenizer2` Initialization

Most of the API for these tokenizers is contained in the `MazeTokenizer2` class. The only time when users need to interact with the internal components of a `MazeTokenizer2` is when initializing a non-default tokenizer.

In [5]:
mt_default = MazeTokenizer2()
mt_ctt = MazeTokenizer2(coord_tokenizer=CoordTokenizers.CTT())
# mt_default, mt_ctt

(MazeTokenizer2-PromptSequencers.AOTP()-CoordTokenizers.UT()-AdjListTokenizers.Coords(intra=True, post=True, walls=False)-TargetTokenizers.Unlabeled(post=False)-PathTokenizers.Coords(post=False),
 MazeTokenizer2-PromptSequencers.AOTP()-CoordTokenizers.CTT(pre=True, intra=True, post=True)-AdjListTokenizers.Coords(intra=True, post=True, walls=False)-TargetTokenizers.Unlabeled(post=False)-PathTokenizers.Coords(post=False))

There are 5 internal components of every `MazeTokenizer2`, each of which subclasses `TokenizerElement`. 

In [21]:
print("\n".join([str(elem) for elem in TokenizerElement.__subclasses__()]))
print('\n')
print("\n".join([str(elem) for elem in mt._tokenizer_elements]))

<class 'maze_dataset.tokenization.maze_tokenizer.CoordTokenizers.CoordTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.AdjListTokenizers.AdjListTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.TargetTokenizers.TargetTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.PathTokenizers.PathTokenizer'>
<class 'maze_dataset.tokenization.maze_tokenizer.PromptSequencers.PromptSequencer'>


PromptSequencers.AOTP()
CoordTokenizers.UT()
AdjListTokenizers.Coords(intra=True, post=True, walls=False)
TargetTokenizers.Unlabeled(post=False)
PathTokenizers.Coords(post=False)


Some `TokenizerElement`s also contain their own initialization arguments. Currently, all of these arguments are `boolean`-typed, but more complex args could be added in the future. The most common arguments across all `TokenizerElement`s are `pre`, `intra`, and `post`, which all control the option to add delimiter tokens to that part of the output. Other args are more specialized; see the docstrings for more details.

# Vocabulary

All instances of `MazeTokenizer2` uses a static vocabulary `VOCAB`, which is one of the main functional differences from `MazeTokenizer`. Direct access to the static vocabulary can be made through 3 constants:
- `VOCAB`: extension of the existing `SPECIAL_TOKENS`
- `VOCAB_LIST: list[str]`
- `VOCAB_TOKEN_TO_INDEX: dict[str, int]`

In [47]:
print("`VOCAB`:")
for i, t in enumerate(VOCAB):
    if i >= 5: break
    print(f"\t{t}")

print("\n`VOCAB_LIST`: list[str]")
for t in VOCAB_LIST[:5]:
    print(f"\t{t}")
    
print("\n`VOCAB_TOKEN_TO_INDEX`: dict[str, int]")
for t in VOCAB_TOKEN_TO_INDEX:
    if VOCAB_TOKEN_TO_INDEX[t] >= 5: break
    print(f"\t{t}:   \t{VOCAB_TOKEN_TO_INDEX[t]}")


`VOCAB`:
	ADJLIST_START
	ADJLIST_END
	TARGET_START
	TARGET_END
	ORIGIN_START

`VOCAB_LIST`: list[str]
	<ADJLIST_START>
	<ADJLIST_END>
	<TARGET_START>
	<TARGET_END>
	<ORIGIN_START>

`VOCAB_TOKEN_TO_INDEX`: dict[str, int]
	<ADJLIST_START>:   	0
	<ADJLIST_END>:   	1
	<TARGET_START>:   	2
	<TARGET_END>:   	3
	<ORIGIN_START>:   	4


## Considerations of Static Vocabulary

- No more rasterized vs uniform indexing
- Fixed max grid size
  - There is now a fixed maximum maze size which is supported.
  - Unique tokens (`CoordTokenizers.UT`): 50x50
  - Coordinate tuple tokens (`CoordTokenizers.CTT`): 128x128
  - Mazes larger than these sizes are not supported
  - There should be fewer compatibility issues with tokenizers using different `max_grid_size` parameters
- Vocabulary access
  - There is now no need to pass around a tokenizer object or any parameter to access its custom vocabulary

## `ALL_TOKENIZERS`

In general, all combinations of `TokenizerElement`s and their arguments will produce a valid and unique `MazeTokenizer2`. However, it is not guaranteed that every possible `MazeTokenizer2` that can be constructed will make practical sense or have been put through testing. `ALL_TOKENIZERS` contains all the tested tokenizers pre-built. For research investigating many different tokenization schemes, one practical way to access them is by looping through `ALL_TOKENIZERS`.

In [25]:
cfg: MazeDatasetConfig = MazeDatasetConfig(
    name="test",
    grid_n=5,
    n_mazes=5,
    maze_ctor=LatticeMazeGenerators.gen_dfs,
)
maze_dataset: MazeDataset = MazeDataset.from_config(
    cfg,
    do_download=False,
    load_local=False,
    do_generate=True,
    save_local=False,
    verbose=True,
    gen_parallel=False,
)

for tokenizer in ALL_TOKENIZERS():
    print(tokenizer)
    print(maze_dataset[0].as_tokens(tokenizer))

trying to get the dataset 'test-g5-n5-a_dfs-h79913'
generating dataset...


generating & solving mazes:   0%|          | 0/5 [00:00<?, ?maze/s]

generating & solving mazes: 100%|██████████| 5/5 [00:00<00:00, 22.22maze/s]

Got dataset test with 5 items. output.cfg.to_fname() = 'test-g5-n5-a_dfs-h79913'
MazeTokenizer2-PromptSequencers.AOTP()-CoordTokenizers.UT()-AdjListTokenizers.Coords(intra=True, post=True, walls=False)-TargetTokenizers.Unlabeled(post=False)-PathTokenizers.Coords(post=False)
['<ADJLIST_START>', '(0,0)', '<-->', '(1,0)', ';', '(2,0)', '<-->', '(3,0)', ';', '(4,1)', '<-->', '(4,0)', ';', '(2,0)', '<-->', '(2,1)', ';', '(1,0)', '<-->', '(1,1)', ';', '(3,4)', '<-->', '(2,4)', ';', '(4,2)', '<-->', '(4,3)', ';', '(0,0)', '<-->', '(0,1)', ';', '(0,3)', '<-->', '(0,2)', ';', '(4,4)', '<-->', '(3,4)', ';', '(4,3)', '<-->', '(4,4)', ';', '(4,1)', '<-->', '(4,2)', ';', '(2,1)', '<-->', '(2,2)', ';', '(1,4)', '<-->', '(0,4)', ';', '(1,2)', '<-->', '(0,2)', ';', '(2,4)', '<-->', '(2,3)', ';', '(4,0)', '<-->', '(3,0)', ';', '(2,2)', '<-->', '(3,2)', ';', '(1,2)', '<-->', '(2,2)', ';', '(1,3)', '<-->', '(0,3)', ';', '(3,2)', '<-->', '(3,3)', ';', '(0,2)', '<-->', '(0,1)', ';', '(3,1)', '<-->', '(3,2)